In [1]:
from dataloader import *
from model import *
from helper import generate_report

In [2]:
data = DataLoader("load_data")

Train dataset
Train dataset loaded: 6000
{'533713007_bf9f3e25b4.jpg': array([0.75768644, 0.29800415, 1.4517018 , ..., 0.8797278 , 1.4616041 ,
       1.1212387 ], dtype=float32), '448252603_7d928c900e.jpg': array([0.17963669, 0.13180107, 0.10692926, ..., 0.1038208 , 0.41851664,
       0.1217608 ], dtype=float32), '2883099128_0b056eed9e.jpg': array([0.12594877, 0.12895155, 0.19055328, ..., 0.59800845, 1.0968809 ,
       0.6271336 ], dtype=float32), '2473738924_eca928d12f.jpg': array([0.58408624, 0.6581098 , 0.9844053 , ..., 0.04409831, 0.17990632,
       0.02335363], dtype=float32), '2124040721_bffc0a091a.jpg': array([0.08133799, 0.57477283, 0.26849335, ..., 0.84083056, 0.3483912 ,
       0.05468456], dtype=float32), '2610447973_89227ff978.jpg': array([0.58145404, 0.2714883 , 0.5982133 , ..., 0.14967628, 0.30119696,
       0.02637993], dtype=float32), '1392272228_cf104086e6.jpg': array([0.3854677 , 0.2732157 , 0.23450656, ..., 0.18462768, 1.1644707 ,
       0.24560145], dtype=float32), '

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
Train images encoded
Time taken in seconds = 268.20609641075134
Photos: train=6000
Loaded: 8092 
save desc


/home/wisla/projects/test-framework/dataloader.py:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if config_passed is "load_data":
/home/wisla/projects/test-framework/dataloader.py:29: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if config_passed["train_images"] is "flickr8k":
/home/wisla/projects/test-framework/dataloader.py:35: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr8k_polish":
/home/wisla/projects/test-framework/dataloader.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k":
/home/wisla/projects/test-framework/dataloader.py:48: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "flickr30k_polish":
/home/wisla/projects/test-framework/dataloader.py:55: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif config_passed["train_images"] is "aide":
/home/wisla/projects/test-framework/datal

NameError: name 'string' is not defined

In [3]:
model=ModelImpl(data)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 51)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 51, 199)      1475983     input_2[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_1[0][0]                    
______________________________________________________________________________________________

In [4]:
model.train()

Epoch 1/2
1182/1182 [==============================] - 281s 236ms/step - loss: 3.5273
Epoch 2/2
1182/1182 [==============================] - 280s 237ms/step - loss: 2.8486


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [5]:
model.evaluate()


tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 9729, 'reflen': 9680, 'guess': [9729, 8729, 7729, 6729], 'correct': [5150, 1687, 420, 118]}
ratio: 1.0050619834709704
Bleu_1: 0.529
Bleu_2: 0.320
Bleu_3: 0.177
Bleu_4: 0.099
computing METEOR score...
METEOR: 0.147
computing Rouge score...
ROUGE_L: 0.381
computing CIDEr score...
CIDEr: 0.215
computing SPICE score...
SPICE: 0.079
computing WMD score...
WMD: 0.087
results/mixed_coco17_flickr8k.json
{'Bleu_1': 0.5293452564497348, 'Bleu_2': 0.3198488601629389, 'Bleu_3': 0.17714892964375045, 'Bleu_4': 0.09936576242927642, 'METEOR': 0.14664073147830178, 'ROUGE_L': 0.38144348743059053, 'CIDEr': 0.2145107435551248, 'SPICE': 0.07913129746038992, 'WMD': 0.08675719285277698}


In [6]:
generate_report(data.config["results_directory"])


 Final results saved to final_results.csv
